Quick analysis

In [ ]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
from matplotlib.colors import LogNorm

import boost_histogram as bh
import pickle as pl

from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
from pygama.dsp.WaveformBrowser import WaveformBrowser

In [ ]:
dg = DataGroup('./cage.json', load=True)
run = 136
cycle = 1481
# str_query = f'run=={run} and skip==False'
str_query = f'cycle=={cycle} and skip==False'
dg.fileDB.query(str_query, inplace=True)
view_cols = ['runtype', 'run', 'cycle', 'startTime', 'runtime', 'threshold']
print(dg.fileDB)

In [ ]:
#get runtime, startime, runtype

runtype_list = np.array(dg.fileDB['runtype'])
runtype = runtype_list[0]

rt_min = dg.fileDB['runtime'].sum()
u_start = dg.fileDB.iloc[0]['startTime']
t_start = pd.to_datetime(u_start, unit='s') # str

In [ ]:
# get scan position

if runtype == 'alp':
    alphaDB = pd.read_hdf('alphaDB.h5')
    scan_pos = alphaDB.loc[alphaDB['run']==run]
    radius = np.array(scan_pos['radius'])[0]
    angle = np.array(scan_pos['source'])[0]
    print(f'Radius: {radius}; Source Angle: {angle}')
# else:

In [ ]:
# Working with dsp or hit files?

# hit = True 
hit = False #ie working with dsp files

In [ ]:
# Working with calibrated or uncalibrated data?

# cal = True #calibrated data
cal = False #uncalibrated data

In [ ]:
# Working in your own directory or with the CAGE_lh5 directory?

user = False # CAGE_lh5 directory
# user = True # hit filesin my personal directory

In [ ]:
# Get relevant files

lh5_dir = dg.lh5_user_dir if user else dg.lh5_dir
# lh5_dir = '/global/homes/g/gothman/projecta/CAGE_lh5_joule'
print(lh5_dir)

# if hit files
if hit == True:
    file_list = lh5_dir + dg.fileDB['hit_path'] + '/' + dg.fileDB['hit_file']
    
else:
    file_list = lh5_dir + dg.fileDB['dsp_path'] + '/' + dg.fileDB['dsp_file']
    
print(f'lh5_dir: {lh5_dir}')
print(f'file list: {file_list}')

In [ ]:
# Create a dataframe

if (cal == True) and (hit == True):
    df = lh5.load_dfs(file_list, ['energy', 'trapEmax', 'trapEftp', 'trapEmax_cal', 'bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
    
elif (cal == True) and (hit == False):
    df = lh5.load_dfs(file_list, ['energy', 'trapEmax', 'trapEftp', 'trapEmax_cal', 'bl','bl_sig','A_10','AoE', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/dsp')

elif (cal == False) and (hit == True):
    df = lh5.load_dfs(file_list, ['energy', 'trapEmax', 'trapEftp', 'bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
    
elif (cal == False) and (hit == False):
    #df = lh5.load_dfs(file_list, ['energy', 'trapEmax', 'trapEftp', 'bl','bl_sig','A_10','AoE', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/dsp')
    df = lh5.load_dfs(file_list, ['energy', 'trapEmax', 'trapEftp', 'bl','bl_sig', 'bl_slope', 'lf_max', 'A_10','AoE', 'dcr', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/dsp')


else:
    print('dont know what to do here! need to specify if working with calibrated/uncalibrated data, or dsp/hit files')

# df_hit = lh5.load_dfs(file_list, ['trapEmax', 'trapEmax_cal', 'bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
# df_hit = lh5.load_dfs(alpha_hit_list, ['trapEmax', 'bl','bl_sig','A_10','ts_sec', 'dcr_raw'], 'ORSIS3302DecoderForEnergy/hit')
# print(df_hit)
# print(df_hit.columns)

# dsc = df[['bl','bl_sig','A_10','AoE', 'dcr_raw', 'dcr_ftp', 'dcr_max']].describe()
# # dsc = df_hit[['bl','bl_sig','A_10','ts_sec', 'dcr_raw']].describe()
# print(dsc)
print(df.columns)



In [ ]:
# lh5_dir = dg.lh5_user_dir #if user else dg.lh5_dir
# print(lh5_dir)
# hit_list = lh5_dir + dg.fileDB['hit_path'] + '/' + dg.fileDB['hit_file']
# print(hit_list)
# df_hit = lh5.load_dfs(hit_list, ['trapEmax', 'trapEmax_cal', 'trapEftp', 'bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
# # df_hit = lh5.load_dfs(alpha_hit_list, ['trapEmax', 'bl','bl_sig','A_10','ts_sec', 'dcr_raw'], 'ORSIS3302DecoderForEnergy/hit')
# # print(df_hit)
# print(df_hit.columns)

# dsc = df_hit[['bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max']].describe()
# # dsc = df_hit[['bl','bl_sig','A_10','ts_sec', 'dcr_raw']].describe()
# print(dsc)

In [ ]:
etype = 'trapEftp'

In [ ]:
from matplotlib import rcParams, cycler
import matplotlib as mpl

plt.rcParams['figure.figsize'] = [14, 10]

waveforms = []
time= []
e_list = []
n_eranges = 50
nwfs= 50
emin = 5000
emax = 15000

eranges = np.linspace(emin, emax, n_eranges) #trying to get about 60 keV energy slices
for e in eranges:
    elo = e-(0.01*e)
    ehi = e+(0.01*e)
    idx = df[etype].loc[(df[etype] >= elo) &
                            (df[etype] <= ehi)].index[:nwfs]
    raw_store = lh5.Store()
    tb_name = 'ORSIS3302DecoderForEnergy/raw'
    lh5_dir = dg.lh5_dir
    raw_list = lh5_dir + dg.fileDB['raw_path'] + '/' + dg.fileDB['raw_file']
    f_raw = raw_list.values[0] # fixme, only works for one file rn
    data_raw, nrows = raw_store.read_object(tb_name, f_raw, start_row=0, n_rows=idx[-1]+1)

    wfs_all = (data_raw['waveform']['values']).nda
    wfs = wfs_all[idx.values, :]
    bl_means = wfs[:,:800].mean(axis=1)
    wf_blsub = (wfs.transpose() - bl_means).transpose()
    ts = np.arange(0, wf_blsub.shape[1]-1, 1)
    super_wf = np.mean(wf_blsub, axis=0)
    wf_max = np.amax(super_wf)
    superpulse = np.divide(super_wf, wf_max)
    waveforms.append(superpulse)
    time.append(ts)
    e_list.append(e)
#     print(wf_max)
#     print(np.mean(wf_blsub[0,:500]))

fig, ax = plt.subplots()
ax = plt.axes()

colors = plt.cm.viridis(np.linspace(0, 1, n_eranges))
print(len(colors))



c = np.arange(0, n_eranges)

norm = mpl.colors.Normalize(vmin=c.min(), vmax=c.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.jet)
cmap.set_array([])



for n in range(n_eranges):
#     print(colors[n][0])
    plt.plot(ts, waveforms[n][:len(waveforms[n])-1], c=cmap.to_rgba(n))
    
cb = fig.colorbar(cmap, ticks=list(eranges))
cb.set_label("Energy", ha = 'right', va='center', rotation=270, fontsize=20)
cb.ax.tick_params(labelsize=18)
    
plt.xlim(3800, 8000)
plt.ylim(0.4, 1.01)
plt.setp(ax.get_xticklabels(), fontsize=18)
plt.setp(ax.get_yticklabels(), fontsize=18)
plt.title(f'Waveforms, {emin}-{emax} trapEftp, {n_eranges} steps', fontsize=20)
plt.xlabel('clock cycles', fontsize=20)
    
plt.show()


